In [5]:
import fitz 
from PIL import Image

def one_to_one_compound_mapping(fa_pixel, fb_pixel, p_pixel):
    q_pixel = int(fa_pixel * (1 - p_pixel) + fb_pixel * p_pixel)
    return q_pixel

def embed_watermark(pdf_path, watermark_path, output_path):

    if not pdf_path.lower().endswith(".pdf"):
        raise ValueError("Input harus berupa berkas PDF.")

    pdf_reader = fitz.open(pdf_path)
    pdf_writer = fitz.open()
    watermark_width, watermark_height =600,250
    watermark_position = (1000,1210)
    watermark_opacity = 0.5

    watermark = Image.open(watermark_path)

    for page_num in range(pdf_reader.page_count):
        page = pdf_reader[page_num] # menyimpan setiap halaman yang sedang dilooping

        # Konversi PDF ke gambar
        image = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
        img = Image.frombytes("RGB", [image.width, image.height], image.samples)

        watermark_resized = watermark.resize((watermark_width, watermark_height), Image.LANCZOS)

        position_x, position_y = watermark_position

        # Ini proses one to one nya
        img = img.convert("RGBA")
        watermark_resized = watermark_resized.convert("RGBA")

        for y in range(watermark_resized.height):
            for x in range(watermark_resized.width):
                alpha_w = watermark_resized.getpixel((x, y))[3] / 255.0 * watermark_opacity
                pixel_img = img.getpixel((position_x + x, position_y + y))

                if isinstance(pixel_img, int):
                    new_pixel = one_to_one_compound_mapping(pixel_img, watermark_resized.getpixel((x, y))[0], alpha_w)
                else:
                    new_pixel = tuple(one_to_one_compound_mapping(pixel_img[i], watermark_resized.getpixel((x, y))[i], alpha_w) for i in range(3))

                img.putpixel((position_x + x, position_y + y), new_pixel)

        # Simpan gambar hasil penyisipan watermark ke dalam file sementara
        watermarked_path = f"temp_watermarked_page_{page_num + 1}.png"
        img.save(watermarked_path, "PNG")

        # Gambar hasil dimasukkan ke dalam PDF
        pdf_writer.insert_page(page_num, width=image.width, height=image.height)
        pdf_writer[page_num].insert_image(fitz.Rect(0, 0, image.width, image.height), filename=watermarked_path)

    # Simpan PDF yang telah berisi watermark
    pdf_writer.save(output_path)
    pdf_reader.close()

if __name__ == "__main__":
    pdf_path = "./TUGAS KIJ.pdf"
    watermark_path = "./IF Horizontal.png"
    output_path = "Watermark.pdf"

    try:
        embed_watermark(pdf_path, watermark_path, output_path)
        print("Proses selesai.")
    except Exception as e:
        print(f"Terjadi kesalahan: {str(e)}")


Proses selesai.


In [8]:
import fitz 
from PIL import Image

def inverse_fb(fb_pixel):
    # Contoh invers sederhana, hanya untuk demonstrasi
    return 255 - fb_pixel  

def one_to_one_compound_mapping(fa_pixel, fb_pixel, p_pixel):
    invers_fb_pixel = inverse_fb(fb_pixel)
    q_pixel = int(invers_fb_pixel * (fa_pixel * p_pixel))
    return q_pixel

def embed_watermark(pdf_path, watermark_path, output_path):

    if not pdf_path.lower().endswith(".pdf"):
        raise ValueError("Input harus berupa berkas PDF.")

    pdf_reader = fitz.open(pdf_path)
    pdf_writer = fitz.open()
    watermark_width, watermark_height =600,250
    watermark_position = (1000,1210)
    watermark_opacity = 0.5

    watermark = Image.open(watermark_path)

    for page_num in range(pdf_reader.page_count):
        page = pdf_reader[page_num] # menyimpan setiap halaman yang sedang dilooping

        # Konversi PDF ke gambar
        image = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
        img = Image.frombytes("RGB", [image.width, image.height], image.samples)

        watermark_resized = watermark.resize((watermark_width, watermark_height), Image.LANCZOS)

        position_x, position_y = watermark_position

        # Ini proses one to one nya
        img = img.convert("RGBA")
        watermark_resized = watermark_resized.convert("RGBA")

        for y in range(watermark_resized.height):
            for x in range(watermark_resized.width):
                alpha_w = watermark_resized.getpixel((x, y))[3] / 255.0 * watermark_opacity
                pixel_img = img.getpixel((position_x + x, position_y + y))

                if isinstance(pixel_img, int):
                    new_pixel = one_to_one_compound_mapping(pixel_img, watermark_resized.getpixel((x, y))[0], alpha_w)
                else:
                    new_pixel = tuple(one_to_one_compound_mapping(pixel_img[i], watermark_resized.getpixel((x, y))[i], alpha_w) for i in range(3))

                img.putpixel((position_x + x, position_y + y), new_pixel)

        # Simpan gambar hasil penyisipan watermark ke dalam file sementara
        watermarked_path = f"temp_watermarked_page_{page_num + 1}.png"
        img.save(watermarked_path, "PNG")

        # Gambar hasil dimasukkan ke dalam PDF
        pdf_writer.insert_page(page_num, width=image.width, height=image.height)
        pdf_writer[page_num].insert_image(fitz.Rect(0, 0, image.width, image.height), filename=watermarked_path)

    # Simpan PDF yang telah berisi watermark
    pdf_writer.save(output_path)
    pdf_reader.close()

if __name__ == "__main__":
    pdf_path = "./TUGAS KIJ.pdf"
    watermark_path = "./IF Horizontal.png"
    output_path = "Watermark2.pdf"

    try:
        embed_watermark(pdf_path, watermark_path, output_path)
        print("Proses selesai.")
    except Exception as e:
        print(f"Terjadi kesalahan: {str(e)}")


Proses selesai.


In [35]:
import PyPDF2
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

def add_watermark(input_pdf, watermark_image, output_pdf):
    watermark = canvas.Canvas('temp_watermark.pdf', pagesize=letter)
    watermark.setFillAlpha(0.4)

    watermark_width, watermark_height = 200, 100 

    # memasukkan gambar watermark ke dalam canvas
    watermark.drawImage(watermark_image, 220, 400, width=watermark_width, height=watermark_height, preserveAspectRatio=True, mask='auto')

    # Canvas nya disimpan kedalam PDF
    watermark.save()

    # Membuka file PDF input
    with open(input_pdf, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        # Mmebuka file temp_watermark.pdf
        with open('temp_watermark.pdf', 'rb') as watermark_file:
            watermark_reader = PyPDF2.PdfReader(watermark_file)

            # Mmebuat PDF baru dengan PDF WRITER
            pdf_writer = PyPDF2.PdfWriter()

            # Iterasi pada setiap halaman dalam PDF asli
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]

                # Setiap halaman pada PDF asli akan dimerge dengan temp_watermark.pdf
                page.merge_page(watermark_reader.pages[0])

                # Menambahkan halaman yang sudah dimerge kedalam file PDF kosong
                pdf_writer.add_page(page)

    # Menyimpan hasil ke outpu_file
    with open(output_pdf, 'wb') as output_file:
        pdf_writer.write(output_file)


if __name__ == "__main__":
    input_pdf_file = "./TUGAS KIJ.pdf"  
    watermark_image_file = "./IF Horizontal.png"  

    # Add Watermark
    add_watermark(input_pdf_file, watermark_image_file, 'Watermark.pdf')